# Bleaching samples

## update sample type 
- everything is healthy or diseased right now, 
-> change to bleached based sampling notes
-> then crosscheck with colonydata?

In [119]:
import numpy as np
import pandas as pd
import os
import re

In [120]:
os.getcwd() 

'/Users/brookesienkiewicz/Documents/Code_notebook/QA_QC/SampleData'

In [121]:
# upload sample and colony data 
sample=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_samples.csv')
colony=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv')
colony.drop(columns="Unnamed: 0",inplace=True)

### Sample Fixes 
- first fix bad formatting/syntax 

In [122]:
# make all species names uniform 
species_list=sample['Species'].unique()
print(species_list)

['OFAV' 'SSID' 'PAST' 'PSTR' 'MCAV' 'OANN' 'DLAB' 'CNAT' 'DL' 'MMEA'
 'PAST ' 'Unknown' 'OFAV ']


In [123]:
# Change OFAV/OANN variations to ORBI
# already did this one 

# sample.loc[:,'Species']=sample['Species'].str.replace('OANN/OFAV?',"ORBI")
# sample.loc[:,'Species']=sample['Species'].str.replace('OFAV/OANN',"ORBI")

# Fix space in PAST in OFAV
sample.loc[:,'Species']=sample['Species'].str.replace('PAST ',"PAST")
sample.loc[:,'Species']=sample['Species'].str.replace('OFAV ',"OFAV")

# Change 'DL' -> DLAB 
sample['Species'] = sample['Species'].replace('\\bDL\\b', 'DLAB', regex=True)

In [124]:
# Check that changes worked and store specie names in list
species_list=sample['Species'].unique()
print(species_list)

['OFAV' 'SSID' 'PAST' 'PSTR' 'MCAV' 'OANN' 'DLAB' 'CNAT' 'MMEA' 'Unknown']


### Add Bleaching sample type 

In [125]:
# search for sampling notes with clp or clb 

#if col contains clb or clp 
# replace status with bleached tissue 
sample.loc[
    sample['Sampling_notes'].str.contains('CLP|CLB', case=False, na=False),'Health_status']= 'Bleached_Tissue'
# (code is in place replacement)

In [126]:
# match sample type to colony statuses
# got from my cbc_metagenomics code

In [127]:
# add colony ID - t# newtagnum species
sample = sample.copy()
sample['TransectNum_str'] = 'T' + sample['TransectNum'].astype(str)
sample['colony_id'] = sample[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
sample.drop(columns='TransectNum_str', inplace=True)

In [128]:
# add colony ID - t# newtagnum species
colony = colony.copy()
colony['TransectNum_str'] = 'T' + colony['TransectNum'].astype(str)
colony['colony_id'] = colony[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
colony.drop(columns='TransectNum_str', inplace=True)

In [129]:
# filter for UML samples
sample_subset=sample[
    (sample['Sample_type'] == 'Core_EtOH') |
    (sample['Sample_type'] == 'Core_RNAlater')
]

In [130]:
## BELOW SCRIPT IS IN PROGRESS 8/20/2025 

In [131]:
# extract condition columns 
cond_cols=['colony_id'] + colony.columns[colony.columns.str.contains('_Condition')].tolist()

In [132]:
# make merged df for everything

# filter colony data for conditions 
small_colony=colony.loc[:,cond_cols]

# and pivot
conditions_long=small_colony.melt(id_vars='colony_id', 
                          var_name='Month_year', 
                          value_name='colony_condition')

# extract month_year from column names to match sample data format
conditions_long['Month_year'] = conditions_long['Month_year'].str.extract(r'(\d+)').astype(int)
# add back in colony data 
cols_keep=['colony_id','Date_InitialTag','Transect','Species','immune_y/n','Date_DocumentedMortality']
colony_filtered=conditions_long.merge(colony[cols_keep],
                                      on='colony_id', 
                                      how='left')

# filter sample data (USING SUBSET FOR NOW) 
filtered_samples=sample_subset.loc[:,('colony_id','Month_year','Health_status','Sampling_notes','Tubelabel_species')] 
# rename column for sample health statuses 
filtered_samples = filtered_samples.rename(columns={'Health_status': 'sample_condition'})

# merge colony and sample dfs by colony id and monthyear
merged = pd.merge(
    colony_filtered,
    filtered_samples,
    on=['colony_id', 'Month_year'],
    how='outer'
)

In [133]:
merged.loc[merged['colony_condition']=='CLP',:].head(5)

,colony_id,Month_year,colony_condition,Date_InitialTag,Transect,Species,immune_y/n,Date_DocumentedMortality,sample_condition,Sampling_notes,Tubelabel_species
41,T1_12_PSTR,112023,CLP,10/15/19,CBC30N,PSTR,y,Healthy,Bleached_Tissue,100% CLP,112023_BEL_CBC_T1_278_PSTR
54,T1_13_PAST,112023,CLP,6/24/19,CBC30N,PAST,y,12/16/23,NaN,NaN,NaN
152,T1_20_PAST,12024,CLP,6/24/19,CBC30N,PAST,n,4/26/24,NaN,NaN,NaN
166,T1_21_PAST,12024,CLP,6/24/19,CBC30N,PAST,y,8/1/24,Healthy,NaN,012024_BEL_CBC_T1_565_PAST
174,T1_21_PAST,92023,CLP,6/24/19,CBC30N,PAST,y,8/1/24,Healthy,No CL,092023_BEL_CBC_T1_172_PAST


In [134]:
# make merged df for everything (not just by species) and use code below 

# if sample id is not na, colony condition is CLP/CLB and sample status is healthy, 
# replace sample status to bleached_tissue 
# then somehow replace original sample data? maybe just by matching tubelabel?

bl_conds = ['CLP','CLB','CLP,CLB','CLB,CLP']
# get rows with no samples 
# rows_na = (merged['sample_ids'].explode().isna().groupby(level=0).any())

# get rows with clp/clb conditions 
rows = (merged['colony_condition'].isin(bl_conds) &
        # exclude rows with no samples 
        merged['Tubelabel_species'].notna())  

# update only those rows with bleached sample status 
merged.loc[rows, 'sample_condition'] = 'Bleached_Tissue'

In [135]:
merged.loc[merged['colony_condition']=='CLP',:].head(5)
# check it worked

,colony_id,Month_year,colony_condition,Date_InitialTag,Transect,Species,immune_y/n,Date_DocumentedMortality,sample_condition,Sampling_notes,Tubelabel_species
41,T1_12_PSTR,112023,CLP,10/15/19,CBC30N,PSTR,y,Healthy,Bleached_Tissue,100% CLP,112023_BEL_CBC_T1_278_PSTR
54,T1_13_PAST,112023,CLP,6/24/19,CBC30N,PAST,y,12/16/23,NaN,NaN,NaN
152,T1_20_PAST,12024,CLP,6/24/19,CBC30N,PAST,n,4/26/24,NaN,NaN,NaN
166,T1_21_PAST,12024,CLP,6/24/19,CBC30N,PAST,y,8/1/24,Bleached_Tissue,NaN,012024_BEL_CBC_T1_565_PAST
174,T1_21_PAST,92023,CLP,6/24/19,CBC30N,PAST,y,8/1/24,Bleached_Tissue,No CL,092023_BEL_CBC_T1_172_PAST


In [136]:
# create function to crosscheck colony health with samples 

def status_match(group):
    # get the condition and list sample statuses of each colony at each monthyear (by row) 
    cond = str(group['colony_condition'].iloc[0])
    # turned into string to prevent crashes from NAs
    statuses = group['sample_condition'].dropna().tolist()
    month = group['Month_year'].iloc[0]
    immune_dates = [112023, 122023, 12024, 22024, 62024, 82024, 122024] 

    # match colony conditions to sample conditions 
    # if colony_condition is healthy, sample = healthy 
    if cond == 'Healthy':
        return all(s == 'Healthy' for s in statuses)

    # if condition is diseased healthy = diseased_margin and diseased_tissue (but rn these are on 2 diff rows)
    elif cond in ['Diseased','Diseased_Other']:
        return all(x in statuses for x in ['Diseased_Tissue', 'Diseased_Margin'])

    # if condition is clp | clb = bleached_tissue 
    elif any(term in cond for term in ['CLP','CLB']):
        return all(x in statuses for x in ['Bleached_Tissue'])
    
    # if condition is Not_visited, sample = NaN
    # if condition is Dead, sample = NaN 
    elif cond in ['Dead', 'Not_Visited','Not_visited','not_visited','NAN']:
        return all(pd.isna(s) for s in group['sample_condition'])

    # if condition is na, sample should also be na 
    elif pd.isna(cond):
        return all(pd.isna(s) for s in group['sample_condition'])

    # exclude False matches if the colony is non-immune during an immune sampling point 
    # list of immune dates 
    
    # if coral is NOT immune during an immune date 
    elif group['immune_y/n'].iloc[0]=='n' and month in immune_dates:
        # it is ok if the sample status is na 
        return all(pd.isna(s) for s in group['sample_condition'])

    ## UGH but this has to exclude T5 and T6 during 01/2024 and 042024 respectively because they were newly tagged then 

    # any mismatches return as false 
    else:
        return False

In [137]:
# crosscheck colony and sample conditions

checks = []
    # check each 'group' (unique combos of colony and monthyear) and store results in 'checks' list
for (colony_id, month), group in merged.groupby(['colony_id', 'Month_year']):
    checks.append({
        'colony_id': colony_id,
        'Species': group['Species'].iloc[0],
        'immune_y/n': group['immune_y/n'].iloc[0],
        'Date_InitialTag': group['Date_InitialTag'].iloc[0],
        'Month_year': month,
            
        # run function &
        # create col containing results of the function that matches statuses
        'Match': status_match(group),
            
        'colony_condition': group['colony_condition'].iloc[0],
        'sample_condition': group['sample_condition'].tolist(),
        'sample_ids': group['Tubelabel_species'].tolist(),
        'mortality_date' : group['Date_DocumentedMortality'].iloc[0]
})
# convert to dataframe
checks_df=pd.DataFrame(checks)

In [138]:
# check mismatches 

false_matches = {}
for specie in species_list:
    # make individual dfs per sp 
    df = checks_df[checks_df['Species']==specie]
    df.head(2)

    # exclude 122024 since no samples were taken 
    df = df[df['Month_year'] != 122024]
    
    # filter for where the match is false or if the colony condition is na and create a new df 
    false = df[(df['Match'] == False) & (~df['colony_condition'].isna())]
    
    # store false matches in new dictionary with species name as key 
    false_matches[specie] = false

In [139]:
species_list

array(['OFAV', 'SSID', 'PAST', 'PSTR', 'MCAV', 'OANN', 'DLAB', 'CNAT',
       'MMEA', 'Unknown'], dtype=object)

In [140]:
false_matches['MCAV']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date
459,T1_8_MCAV,MCAV,n,6/24/19,122022,False,Diseased,[Diseased_Tissue],[122022_BEL_CBC_T1_144_MCAV],9/25/23
717,T2_53_MCAV,MCAV,n,6/22/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1019_MCAV],Healthy
744,T2_55_MCAV,MCAV,y,6/25/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1016_MCAV],Healthy
812,T2_59_MCAV,MCAV,n,6/21/19,52022,False,Diseased,[nan],[nan],Diseased
949,T2_69_MCAV,MCAV,y,6/25/19,92023,False,Healthy,[Bleached_Tissue],[092023_BEL_CBC_T2_176_MCAV],Healthy
1184,T3_14_MCAV,MCAV,y,6/23/19,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T3_1711_MCAV],Healthy
1375,T3_2_MCAV,MCAV,y,6/23/19,12024,False,"Diseased_Other, CLP",[Healthy],[012024_BEL_CBC_T3_633_MCAV],Diseased
1376,T3_2_MCAV,MCAV,y,6/23/19,22024,False,Diseased_Other,"[Bleached_Tissue, Bleached_Tissue]","[022024_BEL_CBC_T3_842_MCAV, 022024_BEL_CBC_T3...",Diseased
1377,T3_2_MCAV,MCAV,y,6/23/19,42024,False,Diseased_Other,[Healthy],[042024_BEL_CBC_T3_956_MCAV],Diseased
1382,T3_2_MCAV,MCAV,y,6/23/19,82024,False,Diseased_Other,[Healthy],[082024_BEL_CBC_T3_1553_MCAV],Diseased


In [141]:
false_matches['PAST']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date
49,T1_13_PAST,PAST,y,6/24/19,112023,False,CLP,[nan],[nan],12/16/23
105,T1_19_PAST,PAST,n,6/21/19,12024,False,CLB,[nan],[nan],4/1/24
108,T1_19_PAST,PAST,n,6/21/19,52022,False,Diseased,[Diseased_Tissue],[052022_BEL_CBC_T1_52_PAST],4/1/24
113,T1_19_PAST,PAST,n,6/21/19,92023,False,CLB,[nan],[nan],4/1/24
131,T1_20_PAST,PAST,n,6/24/19,12024,False,CLP,[nan],[nan],4/26/24
222,T1_2_PAST,PAST,y,6/21/19,12024,False,"CLP,CLB",[nan],[nan],Healthy
863,T2_63_PAST,PAST,y,6/25/19,22024,False,Healthy,[Bleached_Tissue],[022024_BEL_CBC_T2_811_PAST],Healthy
1744,T3_7_PAST,PAST,n,6/23/19,12024,False,CLP,[nan],[nan],4/26/24


In [142]:
false_matches['SSID']
# why do we have some that say not visited but we have samples for..

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date
13,T1_11_SSID,SSID,n,6/24/19,12024,False,Diseased,[nan],[nan],Diseased
15,T1_11_SSID,SSID,n,6/24/19,42024,False,Diseased,[Diseased_Margin],[042024_BEL_CBC_T1_936_SSID],Diseased
21,T1_11_SSID,SSID,n,6/24/19,92023,False,Diseased,[Diseased_Margin],[092023_BEL_CBC_T1_193_SSID],Diseased
79,T1_16_SSID,SSID,n,6/24/19,12024,False,Diseased,[nan],[nan],Diseased
85,T1_16_SSID,SSID,n,6/24/19,62025,False,Diseased,[nan],[nan],Diseased
87,T1_16_SSID,SSID,n,6/24/19,92023,False,Diseased,"[Bleached_Tissue, Bleached_Tissue]","[092023_BEL_CBC_T1_194_SSID, 092023_BEL_CBC_T1...",Diseased
92,T1_17_SSID,SSID,n,6/24/19,12024,False,Diseased,[nan],[nan],Diseased
94,T1_17_SSID,SSID,n,6/24/19,42024,False,"Diseased, CLP","[Diseased_Tissue, Diseased_Margin]","[042024_BEL_CBC_T1_940_SSID, 042024_BEL_CBC_T1...",Diseased
118,T1_1_SSID,SSID,n,6/21/19,12024,False,Diseased,[nan],[nan],Diseased
120,T1_1_SSID,SSID,n,6/21/19,42024,False,Diseased,[Diseased_Margin],[042024_BEL_CBC_T1_937_SSID],Diseased


In [143]:
false_matches['PSTR']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date
728,T2_54_PSTR,PSTR,n,10/13/19,12024,False,CLP,[nan],[nan],Healthy
1731,T3_75_PSTR,PSTR,n,12/3/22,12024,False,CLP,[nan],[nan],Healthy
1733,T3_75_PSTR,PSTR,n,12/3/22,42024,False,Healthy,"[Diseased_Tissue, Diseased_Margin]","[042024_BEL_CBC_T3_965_PSTR, 042024_BEL_CBC_T3...",Healthy
1885,T4_79_PSTR,PSTR,y,12/5/22,92023,False,Healthy,[Bleached_Tissue],[092023_BEL_CBC_T4_93_PSTR],Healthy
1983,T4_98_PSTR,PSTR,y,12/5/22,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T4_1046_PSTR],Healthy
1987,T4_98_PSTR,PSTR,y,12/5/22,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T4_1725_PSTR],Healthy
1989,T4_98_PSTR,PSTR,y,12/5/22,92023,False,Healthy,[Bleached_Tissue],[092023_BEL_CBC_T4_92_PSTR],Healthy


In [144]:
false_matches['OFAV']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date
1085,T2_80_OFAV,OFAV,n,5/24/22,12024,False,CLP,[nan],[nan],Diseased
1528,T3_38_OFAV,OFAV,y,5/20/22,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T3_1692_OFAV],Healthy
1541,T3_39_OFAV,OFAV,y,5/20/22,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T3_1694_OFAV],Healthy
1859,T4_77_OFAV,OFAV,n,12/5/22,92023,False,Healthy,"[Diseased_Tissue, Diseased_Margin]","[092023_BEL_CBC_T4_90_OFAV, 092023_BEL_CBC_T4_...",Healthy
1866,T4_78_OFAV,OFAV,y,12/5/22,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T4_1042_OFAV],Healthy


In [145]:
false_matches['OANN']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date
157,T1_22_OANN,OANN,n,5/21/22,12024,False,Diseased,[nan],[nan],Diseased
173,T1_23_OANN,OANN,n,5/21/22,52022,False,DC,[Healthy],[052022_BEL_CBC_T1_35_OANN],Healthy
505,T2_30_OANN,OANN,n,12/5/22,12024,False,CLP,[nan],[nan],Diseased
2004,T4_99_OANN,OANN,n,12/5/22,122022,False,Diseased,[nan],[nan],Diseased
2678,T6_28_OANN,OANN,n,4/29/24,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T6_1779_OANN],Healthy
2795,T6_36_OANN,OANN,n,4/29/24,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T6_1787_OANN],Healthy


In [146]:
false_matches['DLAB']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date
300,T1_35_DLAB,DLAB,n,5/25/22,12024,False,"CLP,DC",[nan],[nan],Healthy
310,T1_35_DLAB,DLAB,n,5/25/22,122022,False,DC,[Healthy],[122022_BEL_CBC_T1_154_DL],Healthy
1640,T3_69_DLAB,DLAB,n,12/3/22,12024,False,CLP,[nan],[nan],Healthy
1924,T4_93_DLAB,DLAB,n,12/5/22,92023,False,Healthy,"[Bleached_Tissue, Diseased_Margin]","[092023_BEL_CBC_T4_82_DLAB, 092023_BEL_CBC_T4_...",Healthy


In [147]:
false_matches['CNAT']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date


In [148]:
false_matches['MMEA']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date


In [149]:
false_matches['Unknown']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date


In [150]:
# now just have to figure out how to add these changes back into the original df 